# SOLID Principles in Python


1. 단일 책임 원칙 (SRP)
2. 개방-폐쇄 원칙 (OCP)
3. 리스코프 치환 원칙 (LSP)
4. 인터페이스 분리 원칙 (ISP)
5. 의존성 역전 원칙 (DIP)


## 1) 단일 책임 원칙 (SRP)

> **하나의 클래스는 하나의 책임만 가져야 한다**

아래는 SRP를 어긴 코드와, SRP를 준수하도록 개선한 코드 예시입니다.

In [ ]:
# SRP 위반 예시
class FileManager:
    def init(self, file_path):
        self.file_path = file_path

    def read_file(self):
        """파일 읽기"""
        print(f"Reading file: {self.file_path}")

    def write_file(self, content):
        """파일 쓰기"""
        print(f"Writing to file: {self.file_path}")

    def compress_file(self):
        """파일 압축"""
        print(f"Compressing file: {self.file_path}")

    def send_over_network(self, server):
        """네트워크 전송"""
        print(f"Sending {self.file_path} to {server}")

# 이 코드가 SRP를 위반하는 이유
# FileManager는 파일과 관련된 4가지 역할을 동시에 수행하고 있다.

# 파일 읽기/쓰기 (read_file, write_file) → 파일 관리 책임
# 파일 압축 (compress_file) → 파일 압축 책임
# 파일 네트워크 전송 (send_over_network) → 네트워크 전송 책임

# 이렇게 되면
#    파일 압축 방식이 변경될 때 FileManager를 수정해야 함.
#    파일 전송 방식이 변경될 때 FileManager를 수정해야 함.
#    파일 읽기/쓰기 로직이 변경될 때 FileManager를 수정해야 함.
#    즉, 이 클래스는 너무 많은 책임을 가지고 있어 변경의 이유가 너무 많아짐
#    즉 SRP에 위반됨

In [ ]:
# SRP 준수 예시
class FileManager:
    def init(self, file_path):
        self.file_path = file_path

    def read_file(self):
        print(f"Reading file: {self.file_path}")

    def write_file(self, content):
        print(f"Writing to file: {self.file_path}")

class FileCompressor:
    def compress_file(self, file_path):
        print(f"Compressing file: {file_path}")

class NetworkSender:
    def send_over_network(self, file_path, server):
        print(f"Sending {file_path} to {server}")

## 2) 개방-폐쇄 원칙 (OCP)

> **확장에는 열려 있고, 기존 코드 변경에는 닫혀 있어야 한다**

아래는 엔진(Engine) 추상 클래스를 만들고, 이를 상속하는 각각의 엔진을 `CarOCP`에서 주입받아 사용하는 예시입니다.

In [ ]:
# OCP 위반 예시
class CarBadOCP:
    def __init__(self, brand, engine_type: str):
        self.brand = brand
        self.engine_type = engine_type

    def start_car(self):
        print('Starting', self.brand, '...')
        if self.engine_type == "gas":
            print("Gas engine started.")
        elif self.engine_type == "electric":
            print("Electric engine started silently.")
        else:
            print("Unknown engine type.")

    def stop_car(self):
        if self.engine_type == "gas":
            print("Gas engine stopped.")
        elif self.engine_type == "electric":
            print("Electric engine stopped.")
        else:
            print("Unknown engine type.")

# 사용 예
car_gas = CarBadOCP('BMW', "gas")
car_elec = CarBadOCP('Tesla', "electric")

car_gas.start_car()
car_gas.stop_car()

car_elec.start_car()
car_elec.stop_car()


In [3]:
# OCP 준수 예시
from abc import ABC, abstractmethod

class Engine(ABC):
    @abstractmethod
    def start(self):
        pass
    @abstractmethod
    def stop(self):
        pass

class GasEngine(Engine):
    def start(self):
        print('Gas engine started.')
    def stop(self):
        print('Gas engine stopped.')

class ElectricEngine(Engine):
    def start(self):
        print('Electric engine started silently.')
    def stop(self):
        print('Electric engine stopped.')

class CarOCP:
    def __init__(self, brand, engine: Engine):
        self.brand = brand
        self.engine = engine

    def start_car(self):
        print('Starting', self.brand, '...')
        self.engine.start()

    def stop_car(self):
        self.engine.stop()

# 사용 예
car_gas = CarOCP('BMW', GasEngine())
car_elec = CarOCP('Tesla', ElectricEngine())

car_gas.start_car()
car_gas.stop_car()

car_elec.start_car()
car_elec.stop_car()


Starting BMW ...
Gas engine started.
Gas engine stopped.
Starting Tesla ...
Electric engine started silently.
Electric engine stopped.


## 3) 리스코프 치환 원칙 (LSP)

> **자식 클래스는 부모 클래스를 대체 가능해야 한다**

아래 예시는 `Shape` 추상 클래스를 상속한 `Rectangle`과 `Square`를 예시로 LSP를 확인합니다.

In [4]:
#LSP 위반 예시
class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def set_width(self, width):
        self.width = width

    def set_height(self, height):
        self.height = height

    def area(self):
        return self.width * self.height

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)

    def set_width(self, width):
        self.width = self.height = width  # 정사각형은 너비와 높이가 같아야 함

    def set_height(self, height):
        self.width = self.height = height  # 정사각형은 너비와 높이가 같아야 함

# LSP 테스트
def test_area(rect: Rectangle):
    rect.set_width(4)
    rect.set_height(5)
    print(f"{type(rect).__name__} area: {rect.area()}")

rect = Rectangle(2, 3)
sq = Square(2)

test_area(rect)  # 예상 결과: 4 * 5 = 20
test_area(sq)    # 예상 결과: 5 * 5 = 25 (예상과 다름)


Rectangle area: 20
Square area: 25


In [ ]:
class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

class Rectangle(Shape):
    def __init__(self, width, height):
        self._width = width
        self._height = height

    def area(self):
        return self._width * self._height

class Square(Shape):
    def __init__(self, side):
        self._side = side

    def area(self):
        return self._side * self._side

# LSP 테스트
shapes = [
    Rectangle(4, 5),
    Square(5)
]

for s in shapes:
    print(type(s).__name__, 'area:', s.area())


## 4) 인터페이스 분리 원칙 (ISP)

> **클라이언트가 사용하지 않는 메서드에 의존하지 않아야 한다**

필요한 기능별로 인터페이스(추상 클래스)를 나누어, 각 클래스가 진짜로 필요한 기능만 구현하도록 합니다.

In [ ]:
class Flyable(ABC):
    @abstractmethod
    def fly(self):
        pass

class Swimmable(ABC):
    @abstractmethod
    def swim(self):
        pass

class Bird(Flyable):
    def fly(self):
        print('Bird is flying')

class Fish(Swimmable):
    def swim(self):
        print('Fish is swimming')

# 사용
bird = Bird()
fish = Fish()
bird.fly()
fish.swim()


## 5) 의존성 역전 원칙 (DIP)

> **상위 모듈(정책, 추상화)이 하위 모듈(구현)에 의존하지 않도록** 설계

구현체가 아닌 추상 클래스(또는 인터페이스)에 의존하고, 실제 객체는 외부에서 주입(Dependency Injection)받도록 합니다.

In [ ]:
class AbstractEngine(ABC):
    @abstractmethod
    def start(self):
        pass

class SimpleEngine(AbstractEngine):
    def start(self):
        print('SimpleEngine started')

class DIPCar:
    def __init__(self, engine: AbstractEngine):
        self.engine = engine  # 추상에 의존

    def start_car(self):
        self.engine.start()

# 사용 예
engine = SimpleEngine()
dip_car = DIPCar(engine)
dip_car.start_car()


## 마무리

위 예제 코드를 통해, SOLID 5대 원칙을 파이썬으로 간단히 시연해봤습니다.
- **SRP**: 단일 책임
- **OCP**: 개방-폐쇄
- **LSP**: 리스코프 치환
- **ISP**: 인터페이스 분리
- **DIP**: 의존성 역전

실제 프로젝트에서 이 원칙들을 잘 준수하면, 코드 확장성과 유지보수성이 크게 향상됩니다!